# Merge and change primer for subset for MF9

by Pu Zheng

2024.10.05

In [1]:
import os, sys, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import torch
print(os.getpid())
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares")

383134


# Split sublibrary: MF4plus

In [2]:
mf4plus_folder = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus'
mf4plus_probe_file = os.path.join(mf4plus_folder, '2024_10_4_twist_probes.xlsx')
mf4plus_probes = pd.read_excel(mf4plus_probe_file)

In [3]:
mf4plus_probes.head()

,name,sequence,library
0,Ptprb_ENSMUST00000218553.2_shift:592_GC:56.6_T...,CCCGCAATGGCTGACAACCGGGTGTAGTTCTCCGATTGCGTACGGT...,MF9-mf4plus
1,Ptprb_ENSMUST00000218553.2_shift:645_GC:60.0_T...,CCCGCAATGGCTGACAACCGACGCACGCTCTCTTGCCGTCTGGTGT...,MF9-mf4plus
2,Ptprb_ENSMUST00000218553.2_shift:980_GC:56.6_T...,CCCGCAATGGCTGACAACCGACGGTCCACGTCCCACGACGTGGTGT...,MF9-mf4plus
3,Ptprb_ENSMUST00000218553.2_shift:1087_GC:43.3_...,CCCGCAATGGCTGACAACCGGGTGTAGTTCTCCGATTGCGTACGCA...,MF9-mf4plus
4,Ptprb_ENSMUST00000218553.2_shift:1800_GC:46.6_...,CCCGCAATGGCTGACAACCGGGTGTAGTTCTCCGATTGCGTACGCA...,MF9-mf4plus


In [4]:
manual_genes = pd.read_csv(os.path.join(mf4plus_folder, 'mf4plus_base.csv'))['Gene'].values
lung_genes = pd.read_csv(os.path.join(mf4plus_folder, 'manual_lung_genes.csv'))['Gene'].values


In [5]:
len(manual_genes), len(lung_genes)

(150, 25)

In [6]:
mf4plus_probes['gene'] = [_n.split('_')[0] for _n in mf4plus_probes['name']]

In [7]:
mf4plus_base_probes = mf4plus_probes.loc[[_g in manual_genes for _g in mf4plus_probes['gene']]]

In [8]:
mf4plus_addon_probes = mf4plus_probes.loc[[_g in lung_genes for _g in mf4plus_probes['gene']]]

In [9]:
# swap new primers:
primer_len = 20
old_primers = pd.read_csv(os.path.join(mf4plus_folder, 'selected_primers.csv'), index_col=0)
old_primers


,id,description,sequence,sequence_rc,sequence_OT,sequence_rc_OT
0,W1A03_primer_2,W1A03_primer_2,CCCGCAATGGCTGACAACCG,CGGTTGTCAGCCATTGCGGG,0,0
1,W1A10_primer_9,W1A10_primer_9,TAATACGACTCACTATAGGGATTGCCGCATGGTTTCCG,CGGAAACCATGCGGCAATCCCTATAGTGAGTCGTATTA,0,0


In [10]:
mf4plus_addon_folder = r'/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus_addon'
new_primers = pd.read_csv(os.path.join(mf4plus_addon_folder, 'selected_primers.csv'), index_col=0)
new_primers

,id,description,sequence,sequence_rc,sequence_OT,sequence_rc_OT
0,W1B05_primer_16,W1B05_primer_16,CGGGTTTCGTTGCGCACACC,GGTGTGCGCAACGAAACCCG,0,0
1,W1B04_primer_15,W1B04_primer_15,TAATACGACTCACTATAGGGCTTGTGCATCGCGCCAAAGA,TCTTTGGCGCGATGCACAAGCCCTATAGTGAGTCGTATTA,0,0


In [11]:
new_seqs = [] #, new_names = [], []
for _i, _row in mf4plus_addon_probes.iterrows():
    _seq, _name = _row['sequence'], _row['name']
    if old_primers.loc[0, 'sequence'][-primer_len:] not in _seq:
        raise ValueError("Fwd primer not found")
    elif old_primers.loc[1, 'sequence_rc'][:primer_len] not in _seq:
        raise ValueError("Rev primer not found")
    # replace:
    new_seqs.append(_seq.replace(old_primers.loc[0, 'sequence'][-primer_len:],
                                 new_primers.loc[0, 'sequence'][-primer_len:]).replace(old_primers.loc[1, 'sequence_rc'][:primer_len],
                                                                                        new_primers.loc[1, 'sequence_rc'][:primer_len]))
# replace this column:
mf4plus_addon_probes.loc[:,'sequence'] = new_seqs
# change name:
mf4plus_addon_probes.loc[:,'library'] = 'MF9-mf4plus-addon'
# save:
mf4plus_addon_savefile = os.path.join(mf4plus_addon_folder, f"{time.localtime().tm_year}_{time.localtime().tm_mon}_{time.localtime().tm_mday}_mf4plus_addon_twist_probes.xlsx")
print(mf4plus_addon_savefile)
mf4plus_addon_probes.to_excel(mf4plus_addon_savefile, index=False)

/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus_addon/2024_10_6_mf4plus_addon_twist_probes.xlsx


In [12]:
# Save the mf4_base:
mf4plus_base_probes.loc[:,'library'] = 'MF9-mf4plus-base'
mf4plus_base_savefile = os.path.join(mf4plus_folder, f"{time.localtime().tm_year}_{time.localtime().tm_mon}_{time.localtime().tm_mday}_mf4plus_base_twist_probes.xlsx")
print(mf4plus_base_savefile)
mf4plus_base_probes.to_excel(mf4plus_base_savefile, index=False)

/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus/2024_10_6_mf4plus_base_twist_probes.xlsx


# Merge:

In [13]:
# probes to merge:
# Manually pasted this list!
subpool_probe_files = [
    '/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus_addon/2024_10_6_mf4plus_addon_twist_probes.xlsx',
    '/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus/2024_10_6_mf4plus_base_twist_probes.xlsx',
    '/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf7minus/2024_10_4_twist_probes.xlsx',
]
# load
probe_list = [pd.read_excel(_f) for _f in subpool_probe_files]
for _f, _pbs in zip(subpool_probe_files, probe_list):
    print(_f, len(_pbs))

/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus_addon/2024_10_6_mf4plus_addon_twist_probes.xlsx 2364
/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus/2024_10_6_mf4plus_base_twist_probes.xlsx 14144
/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf7minus/2024_10_4_twist_probes.xlsx 24800


In [14]:
#concat:
merged_probes = pd.concat(probe_list)

In [22]:
len(merged_probes)

41308

In [17]:
# check library tag
print(np.unique(merged_probes['library'], return_counts=True))

(array(['MF9-mf4plus-addon', 'MF9-mf4plus-base', 'MF9-mf7minus'],
      dtype=object), array([ 2364, 14144, 24800]))


In [18]:
# check primer:
merged_probes.loc[:,'fwd_primer'] = [_s[:primer_len] for _s in merged_probes['sequence']]
merged_probes.loc[:,'rev_primer'] = [_s[-primer_len:] for _s in merged_probes['sequence']]

In [19]:
print(np.unique(merged_probes['fwd_primer'], return_counts=True))

(array(['CCCGCAATGGCTGACAACCG', 'CGCAAACTGGTGCGGAAGGC',
       'CGGGTTTCGTTGCGCACACC'], dtype=object), array([14144, 24800,  2364]))


In [20]:
print(np.unique(merged_probes['rev_primer'], return_counts=True))

(array(['CGGAAACCATGCGGCAATCC', 'CTCGACCTCGCGGGCAATGG',
       'TCTTTGGCGCGATGCACAAG'], dtype=object), array([14144, 24800,  2364]))


## all checks out, save now: 

In [21]:
merged_savefile = os.path.join(mf4plus_addon_folder, f"{time.localtime().tm_year}_{time.localtime().tm_mon}_{time.localtime().tm_mday}_mergedMF9_twist_probes.xlsx")
print(merged_savefile)
merged_probes.to_excel(merged_savefile, index=False)

/lab/solexa_weissman/puzheng/MERFISH_Probes/4T1_MERFISH/MF9_4T1Lung_mf4plus_addon/2024_10_6_mergedMF9_twist_probes.xlsx
